In [1]:
import dask.dataframe as dd
import pandas as pd
import os
import time
from dask.distributed import Client, LocalCluster

In [2]:
import numpy as np

In [3]:
from dask_ml.preprocessing import Categorizer, DummyEncoder

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
os.chdir('F:\\User Data\\Sandeep Menon\\Colorado State Targeting')

In [6]:
Control_States = dd.read_csv("Random_Forest_Input_Clean_Py5.csv",assume_missing=True,dtype={'TDLINX_OUTLET_CODE': 'object','TDLINX_MODULE_CHANGE_CODE': 'object','TDLINX_SUB_CHANNEL_CODE': 'object'})

In [7]:
len(Control_States)

361635

In [8]:
Control_States.head()

,TDLINX_OUTLET_CODE,Product,Sum_RAD_QTY_9L,Status,TradeAreaVolume_RAD_QTY_9L,TradeAreaBVVolume,Female-21,Female-65,Male-21,Male-65,...,AvgAwayExp-Wine,AvgAwayExp-Other,AvgExp-OnTrips,Mosaic,BRAND,Age 21-34,Age 34+,Income Low,Income Mid,Income High
0,0109172,Ciroc Red Berry Vodka 0.05L,0.0,New,163498.567907,0.0,341165.0,60313.0,341181.0,46443.0,...,36.85,36.86,61.90,Group O Singles and Starters,178.0,219608.0,355981.0,148317.0,176700.0,42275.0
1,0110025,Ciroc Red Berry Vodka 0.05L,0.0,New,95883.430747,0.0,209610.0,42533.0,208718.0,32891.0,...,36.09,35.27,60.72,Group O Singles and Starters,178.0,126464.0,216437.0,92688.0,105812.0,25010.0
2,0110350,Ciroc Red Berry Vodka 0.05L,0.0,New,62112.688735,0.0,165449.0,30980.0,161411.0,22821.0,...,32.01,34.60,55.37,Group O Singles and Starters,178.0,111295.0,161764.0,84031.0,86327.0,11959.0
3,0110506,Ciroc Red Berry Vodka 0.05L,0.0,New,1184.438875,0.0,3661.0,1143.0,3769.0,925.0,...,26.52,26.03,45.26,Group J Autumn Years,178.0,1675.0,3686.0,2551.0,1470.0,93.0
4,0110586,Ciroc Red Berry Vodka 0.05L,0.0,New,3378.383316,0.0,11851.0,3921.0,15649.0,3311.0,...,31.32,30.23,53.67,Group Q Golden Year Guardians,178.0,6451.0,13819.0,7258.0,4998.0,484.0


In [9]:
Control_States['TDLINX_OUTLET_CODE']=Control_States['TDLINX_OUTLET_CODE'].astype('object')

In [10]:
Control_States.dtypes

TDLINX_OUTLET_CODE             object
Product                        object
Sum_RAD_QTY_9L                float64
Status                         object
TradeAreaVolume_RAD_QTY_9L    float64
TradeAreaBVVolume             float64
Female-21                     float64
Female-65                     float64
Male-21                       float64
Male-65                       float64
Asian 21                      float64
Asian 65                      float64
AfAm 21                       float64
AfAm 65                       float64
Hisp 21                       float64
Hisp 65                       float64
White 21                      float64
White 65                      float64
AvgHomeExp-Beer               float64
AvgHomeExp-Whiskey            float64
AvgHomeExp-Wine               float64
AvgHomeExp-Others             float64
AvgAwayExp-Beer               float64
AvgAwayExp-Wine               float64
AvgAwayExp-Other              float64
AvgExp-OnTrips                float64
Mosaic      

In [11]:
df_category = Control_States.select_dtypes(include=['object'])

In [12]:
df_float = Control_States.select_dtypes(include=['float64'])

In [13]:
len(df_category.columns)

4

In [14]:
df_category_columns = list(df_category.columns)

In [15]:
for col in df_category_columns:
    Control_States[col]=Control_States[col].astype('category')

In [16]:
BV_List = set(list(Control_States['Product'].compute()))

In [17]:
len(BV_List)

57

In [18]:
n=len(BV_List)

In [19]:
df_category.columns

Index(['TDLINX_OUTLET_CODE', 'Product', 'Status', 'Mosaic'], dtype='object')

In [20]:
#df_category_model = df_category.drop(['TDLINX_OUTLET_CODE','INDIVIDUAL_BRAND_SIZE_DESC','TDLINX_OUTLET_DESC','TDLINX_OUTLET_CITY',
#                                   'PLACE_DESC','COUNTY_DESC','COUNTRY_DESC','TDLINX_SECTOR_DESC','TDLINX_SUB_CHANNEL_DESC',
#                                    'TDLINX_CHAIN_IND_DESC','TDLINX_FOOD_TYPE_DESC','SPIRITS_NRI_CATEGORY_DESC','TDLINX_OUTLET_STATUS_DESC'],axis=1)

In [21]:
df_category_model = df_category.drop(['TDLINX_OUTLET_CODE','Mosaic'],axis=1)

In [22]:
#BV = df_category_model['INDIVIDUAL_BRAND_DESC'].compute()
BV1 = df_category_model.loc[:,'Product']

In [23]:
#type(BV)
type(BV1)

dask.dataframe.core.Series

In [24]:
df_float = df_float.drop(['BRAND'],axis=1)

In [25]:
df_category_model = df_category_model.categorize()

In [26]:
df_category_model.dtypes

Product    category
Status     category
dtype: object

In [27]:
de = DummyEncoder()
trn = de.fit_transform(df_category_model)

In [28]:
trn.head()

,Product_Ciroc Red Berry Vodka 0.05L,Product_Ciroc Red Berry Vodka 1.0L,Product_Ciroc Red Berry Vodka 1.75L,Product_Ciroc Summer Colada 0.375L,Product_Ciroc Summer Colada 0.75L,Product_Ciroc Summer Colada 1.0L,Product_Ciroc Ten,Product_Ciroc Vodka,Product_Ciroc Vodka 0.05L,Product_Ciroc Vodka 1.75L,...,Product_JW Blue Label 0.2L,Product_JW Blue Label 0.75L,Product_JW Blue Label Ghost and Rare,Product_JW Blue Label Ghost and Rare 0.75L,Product_JW Double Black,Product_JW Red Label 0.2L,Product_JW Red Label 0.375L,Product_JW The Collection 0.2L,Status_New,Status_Existing
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [29]:
df_float.dtypes

Sum_RAD_QTY_9L                float64
TradeAreaVolume_RAD_QTY_9L    float64
TradeAreaBVVolume             float64
Female-21                     float64
Female-65                     float64
Male-21                       float64
Male-65                       float64
Asian 21                      float64
Asian 65                      float64
AfAm 21                       float64
AfAm 65                       float64
Hisp 21                       float64
Hisp 65                       float64
White 21                      float64
White 65                      float64
AvgHomeExp-Beer               float64
AvgHomeExp-Whiskey            float64
AvgHomeExp-Wine               float64
AvgHomeExp-Others             float64
AvgAwayExp-Beer               float64
AvgAwayExp-Wine               float64
AvgAwayExp-Other              float64
AvgExp-OnTrips                float64
Age 21-34                     float64
Age 34+                       float64
Income Low                    float64
Income Mid  

In [30]:
#df_float = df_float.drop(['LATITUDE','LONGITUDE','SELLING_SPACE_AREA'],axis=1)

In [31]:
df_float.head()

,Sum_RAD_QTY_9L,TradeAreaVolume_RAD_QTY_9L,TradeAreaBVVolume,Female-21,Female-65,Male-21,Male-65,Asian 21,Asian 65,AfAm 21,...,AvgHomeExp-Others,AvgAwayExp-Beer,AvgAwayExp-Wine,AvgAwayExp-Other,AvgExp-OnTrips,Age 21-34,Age 34+,Income Low,Income Mid,Income High
0,0.0,163498.567907,0.0,341165.0,60313.0,341181.0,46443.0,23754.0,2944.0,25720.0,...,17.02,100.11,36.85,36.86,61.90,219608.0,355981.0,148317.0,176700.0,42275.0
1,0.0,95883.430747,0.0,209610.0,42533.0,208718.0,32891.0,10559.0,1613.0,6642.0,...,16.85,94.75,36.09,35.27,60.72,126464.0,216437.0,92688.0,105812.0,25010.0
2,0.0,62112.688735,0.0,165449.0,30980.0,161411.0,22821.0,10100.0,1352.0,24283.0,...,15.62,91.21,32.01,34.60,55.37,111295.0,161764.0,84031.0,86327.0,11959.0
3,0.0,1184.438875,0.0,3661.0,1143.0,3769.0,925.0,95.0,11.0,109.0,...,13.24,58.23,26.52,26.03,45.26,1675.0,3686.0,2551.0,1470.0,93.0
4,0.0,3378.383316,0.0,11851.0,3921.0,15649.0,3311.0,192.0,18.0,870.0,...,15.22,74.28,31.32,30.23,53.67,6451.0,13819.0,7258.0,4998.0,484.0


In [32]:
Final_Control_States = dd.concat([df_float, trn, BV1], axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\dask\dataframe\multi.py:602: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indexes of each dataframes are 
aligned. This assumption is not generally safe.
  warn("Concatenating dataframes with unknown divisions.\n"


In [33]:
Final_Control_States.dtypes

Sum_RAD_QTY_9L                                    float64
TradeAreaVolume_RAD_QTY_9L                        float64
TradeAreaBVVolume                                 float64
Female-21                                         float64
Female-65                                         float64
Male-21                                           float64
Male-65                                           float64
Asian 21                                          float64
Asian 65                                          float64
AfAm 21                                           float64
AfAm 65                                           float64
Hisp 21                                           float64
Hisp 65                                           float64
White 21                                          float64
White 65                                          float64
AvgHomeExp-Beer                                   float64
AvgHomeExp-Whiskey                                float64
AvgHomeExp-Win

In [34]:
#Final_Control_States = Final_Control_States.drop(['BV_TradeAreaVolume'],axis=1)

In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import train_test_split

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [36]:
BV_List = list(BV_List)

In [37]:
BV_List

['Don Julio 70 Anejo 0.75L',
 'George Dickel Tabasco Barrel Finish',
 'DeLeon Platinum',
 'DeLeon Platinum 0.05L',
 'JW Double Black',
 'Ciroc Vodka 0.05L',
 'Cragganmore 12YO',
 'DeLeon Platinum 0.375L',
 'JW Black Label 0.05L',
 'Don Julio 1942',
 'DeLeon Platinum 1.75L',
 'Dalwhinnie 25YO 0.75L',
 'JW Aged 18 Years 0.2L',
 'Ciroc Vodka',
 'Crown Royal Honey',
 'DeLeon Extra Anejo 0.75L',
 'DeLeon Platinum 0.75L',
 'JW Blue Label 0.2L',
 'JW Blue Label Ghost and Rare',
 'JW Aged 18 Years 0.75L',
 'Ciroc Summer Colada 0.75L',
 'Crown Royal Canadian Whiskey 0.05L',
 'Ciroc Summer Colada 1.0L',
 'Crown Royal Maple Finished',
 'Crown Royal Canadian Whiskey 0.75L',
 'DeLeon Reposado 0.375L',
 'George Dickel Barrel Select 0.75L',
 'JW Blue Label 0.75L',
 'Crown Royal Northern Harvest Rye 0.05L',
 'Crown Royal Vanilla 1.75L',
 'JW Blue Label Ghost and Rare 0.75L',
 'Don Julio 1942 1.75L',
 'Ciroc Vodka 1.75L',
 'Don Julio Real',
 'DeLeon Reposado',
 'Ciroc Summer Colada 0.375L',
 'Dalwhinni

In [38]:
#Final_Control_States = Final_Control_States.drop(['LONGITUDE'],axis=1)

In [39]:
Final_Control_States_Existing = Final_Control_States[Final_Control_States['Status_Existing']==1]

In [40]:
Final_Control_States_New = Final_Control_States[Final_Control_States['Status_New']==1]

In [41]:
Control_States_Existing = Control_States[Control_States['Status']=="Existing"]

In [42]:
Control_States_New = Control_States[Control_States['Status']=="New"]

In [43]:
import dask_ml.joblib
from dask.distributed import Client
from sklearn.externals import joblib
from dask.distributed import Client, LocalCluster

In [44]:
cluster = LocalCluster()
client= Client()

In [45]:
client

Client Scheduler: tcp://127.0.0.1:56852 Dashboard: http://127.0.0.1:56853/status,Cluster Workers: 32 Cores: 32 Memory: 137.44 GB


In [46]:
import dask.distributed
c = dask.distributed.Client()

In [47]:
#Pandas dataframe Approach
i = 0
prediction_set = pd.DataFrame()
prediction = pd.DataFrame()
prediction1_set = pd.DataFrame()
Actual_Set = pd.DataFrame()
Actual1_Set = pd.DataFrame()
outlet_code = pd.DataFrame()
outlet1_code =pd.DataFrame()
predictor = pd.DataFrame()
BV = pd.DataFrame()
BV1 = pd.DataFrame()
for i in range(0,n):
        df = Final_Control_States_Existing
        df1 = Final_Control_States_New
        df = df.loc[df['Product'] == BV_List[i]]
        df1 = df1.loc[df1['Product']== BV_List[i]]
        bv = df.loc[:,'Product']
        bv = bv.to_frame()
        bv = bv.compute()
        bv1 = df1.loc[:,'Product']
        bv1 = bv1.to_frame()
        bv1 = bv1.compute()
        df_outlet = Control_States_Existing
        df1_outlet = Control_States_New
        df_outlet = df_outlet.loc[df_outlet['Product'] == BV_List[i]]
        df1_outlet = df1_outlet.loc[df1_outlet['Product'] == BV_List[i]]
        outlet = df_outlet.loc[:,'TDLINX_OUTLET_CODE']
        outlet = outlet.to_frame()
        outlet = outlet.compute()
        outlet1 = df1_outlet.loc[:,'TDLINX_OUTLET_CODE']
        outlet1 = outlet1.to_frame()
        outlet1 = outlet1.compute()
        X =  df.drop(['Sum_RAD_QTY_9L','Product'],axis=1)
        X1 = df1.drop(['Sum_RAD_QTY_9L','Product'],axis =1)
        y =  df.loc[:,'Sum_RAD_QTY_9L']
        y =  y.to_frame()
        y = y.compute()
        y1 = df1.loc[:,'Sum_RAD_QTY_9L']
        y1 = y1.to_frame()
        y1 = y1.compute()
        start_time = time.time()
        est = RandomForestRegressor(n_estimators=100,max_features="auto",max_depth=15)
        Model=est.fit(X,y)
        print(i)
        print(str(BV_List[i]))
        print(str(len(X))+" " +"NO_Rows_in_training")
        print(str(len(X1))+" " +"NO_Rows_in Validation")
        R2 = Model.score(X,y)
        R2_Validation = Model.score(X1,y1)
        print(str(R2) + "R2_Training")
        print(str(R2_Validation) + "R2_validation" )
        print(str(time.time() - start_time)+" "+ "seconds" )
        prediction = pd.DataFrame(Model.predict(X))
        prediction1 = pd.DataFrame(Model.predict(X1))
        prediction_set = prediction_set.append(prediction)
        prediction1_set = prediction1_set.append(prediction1)
        Actual_Set = Actual_Set.append(y)
        Actual1_Set = Actual1_Set.append(y1)
        outlet_code = outlet_code.append(outlet)
        outlet1_code = outlet1_code.append(outlet1)
        BV = BV.append(bv)
        BV1 = BV1.append(bv1)
        

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


0
Don Julio 70 Anejo 0.75L
466 NO_Rows_in_training
5878 NO_Rows_in Validation
0.8441535394287298R2_Training
0.0R2_validation
19.65712881088257 seconds


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


1
George Dickel Tabasco Barrel Finish
425 NO_Rows_in_training
5922 NO_Rows_in Validation
0.7677334909427185R2_Training
0.0R2_validation
19.235128164291382 seconds


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


...

54
Crown Royal Salted Caramel 0.75L
712 NO_Rows_in_training
5632 NO_Rows_in Validation
0.8405397818048376R2_Training
0.0R2_validation
20.581767559051514 seconds


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


55
Crown Royal Texas Mesquite Smoked Whisky 0.75L
479 NO_Rows_in_training
5865 NO_Rows_in Validation
0.8508942004568977R2_Training
0.0R2_validation
19.970080852508545 seconds


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


56
Don Julio 1942 0.75L
581 NO_Rows_in_training
5763 NO_Rows_in Validation
0.813658317887722R2_Training
0.0R2_validation
20.87669062614441 seconds


In [48]:
#df_outlet = df_outlet.to_frame()
#df_outlet = df_outlet.compute()
#df_outlet = df_outlet.reset_index(drop=True)

In [49]:
#df_outlet = Control_States_Existing.loc[:,'TDLINX_OUTLET_CODE']

In [50]:
#panda = Final_Control_States_Existing.compute()
#panda = panda.reset_index(drop=True)

In [51]:
#Data = pd.concat([df_outlet,panda],axis=1)

In [52]:
#Train_Test_Validation:
#i=0
#Y_test = pd.DataFrame()
#X_Test = pd.DataFrame()
#outlet_code_train = pd.DataFrame()
#outlet_code_test = pd.DataFrame()
#BV_train = pd.DataFrame()
#BV_test = pd.DataFrame()
#Prediction_Test = pd.DataFrame()
##Data = pd.concat([Control_States_Existing.loc[:,'TDLINX_OUTLET_CODE'],Final_Control_States_Existing],axis=1)
#for i in range(0,5):
#    df = Data
#    df = df.loc[df['INDIVIDUAL_BRAND_DESC'] == BV_List[i]]
#    X =  df.drop(['BV_Depletions'],axis=1)
#    y =  df.loc[:,'BV_Depletions']
#    X_train, X_test, y_train, y_test = train_test_split(
#              X, y, test_size=0.33, random_state=42)
#    #X_test = X_test.to_frame()
#    #X_test = X_test.compute()
#    y_test = y_test.to_frame()
#    #y_test = y_test.compute()
#    outlet = X_test.loc[:,'TDLINX_OUTLET_CODE']
#    bv =     X_test.loc[:,'INDIVIDUAL_BRAND_DESC']
#    X_test = X_test.drop(['TDLINX_OUTLET_CODE'],axis=1)
#    X_train = X_train.drop(['TDLINX_OUTLET_CODE'],axis=1)
#    X_test = X_test.drop(['INDIVIDUAL_BRAND_DESC'],axis=1)
#    X_train = X_train.drop(['INDIVIDUAL_BRAND_DESC'],axis=1)
#    print(str(len(X_train))+ "no rows in X_train")
#    est = RandomForestRegressor(n_estimators=70,max_features=0.4,min_samples_split=50)
#    Model=est.fit(X_train,y_train)
    #outlet = outlet.to_frame()
    #score = Model.score(X_test,y_test)
    #print(score)
    #prediction = Model.predict(X_test)
    #prediction = pd.DataFrame(prediction)
    ##prediction = prediction.to_frame()
    ##outlet = outlet.compute()
    #X_Test = X_Test.append(X_test)
    #Y_test = Y_test.append(y_test)
    #bv = bv.to_frame()
    #BV_test = BV_test.append(bv)
    #Prediction_Test = Prediction_Test.append(prediction)
    
    


In [53]:
#X_test.columns

In [54]:
BV= BV.reset_index(drop=True)
outlet_code = outlet_code.reset_index(drop=True)
Actual_Set = Actual_Set.reset_index(drop=True)
prediction_set = prediction_set.reset_index(drop=True)
Existing = pd.concat([BV,outlet_code,Actual_Set,prediction_set],axis=1)

In [55]:
BV1= BV1.reset_index(drop=True)
outlet1_code = outlet1_code.reset_index(drop=True)
Actual1_Set = Actual1_Set.reset_index(drop=True)
prediction1_set = prediction1_set.reset_index(drop=True)
New = pd.concat([BV1,outlet1_code,Actual1_Set,prediction1_set],axis=1)

In [56]:
#Existing = pd.concat([BV,outlet_code,Actual_Set,prediction_set],axis=1)

In [57]:
#New = pd.concat([BV1,outlet1_code,Actual1_Set,prediction1_set],axis=1)

In [58]:
Existing.to_csv('Existing_BVS1e.csv',header=True,index=False)

In [59]:
New.to_csv('New_BVS1e.csv',header=True,index=False)

In [60]:
New_Existing_Combined = Existing.append(New)

In [61]:
New_Existing_Combined.to_csv('New_Existing_e.csv',header=True,index=False)

In [62]:
#for col in X.columns:
   # k = np.isnan(X[col]).sum().compute()
    #print(col)
    #print(k)

In [63]:
#y.isnull().sum().compute()

In [64]:
#np.all(np.isfinite(X))

In [65]:
#start_time = time.time()
#est = RandomForestRegressor(n_estimators=100,max_features="auto")
#Model=est.fit(X,y)
#print(str(time.time() - start_time)+" "+ "seconds" )